In [ ]:
import pandas as pd
from datetime import datetime
pd.options.plotting.backend = "plotly"

zelty = pd.read_csv("../zelty/zelty.csv", sep=";",decimal=',',low_memory=False, index_col=False)
zelty['date'] = pd.to_datetime(zelty['date'])
zelty.head()

In [ ]:
adhésions = zelty[zelty['category'] == 'Adhésion'].groupby('date')[['price']].count()

In [ ]:
limit_date = datetime.now().replace(month=datetime.now().month - 2)
adhésions_last_month = adhésions[adhésions.index.to_period("M") >= str(limit_date.year)+'-'+str(limit_date.month)]
adhésions_last_month = adhésions_last_month.resample('D').sum().rename(columns={'price': 'nb'})
adhésions_last_month.plot(kind="bar")

In [ ]:
limit_date = datetime.now().replace(year=datetime.now().year - 1)
adhésions_top_year = adhésions[adhésions.index.to_period("M") >= str(limit_date.year)+'-'+str(limit_date.month)]
adhésions_top_year = adhésions_top_year.resample('D').sum().rename(columns={'price': 'nb'})
adhésions_top_year.sort_values(by='nb', ascending=False).head(20)

In [ ]:
adhésions_by_month = adhésions.resample('M').sum().rename(columns={'price': 'nb'})
adhésions_by_month['year'] = adhésions_by_month.index.year
adhésions_by_month = adhésions_by_month.groupby(['date','year']).sum().unstack('year').fillna(0)
adhésions_by_month.columns = adhésions_by_month.columns.droplevel(0)
adhésions_by_month.plot(kind='bar')


In [ ]:
#adhésions_by_year.plot(kind="bar")

In [ ]:
adhésions_by_year = adhésions.resample('Y').sum().rename(columns={'price': 'nb'})
adhésions_by_year.index = adhésions_by_year.index.year

adhésions_a_date = adhésions.copy()
adhésions_a_date['a_date'] = adhésions_a_date.index.month*100+adhésions_a_date.index.day
adhésions_a_date = adhésions_a_date[adhésions_a_date['a_date'] < datetime.now().month*100+datetime.now().day]
del adhésions_a_date['a_date']
adhésions_a_date = adhésions_a_date.resample('Y').sum().rename(columns={'price': 'nb'})
adhésions_a_date.index = adhésions_a_date.index.year
libelle_a_date = "à date du %02d/%02d" % (datetime.now().day, datetime.now().month)
adhésions_by_year[libelle_a_date] = adhésions_a_date['nb']
#adhésions_by_year.histogram()
adhésions_by_year.fillna(0, inplace=True)
adhésions_by_year['toute l\'année'] = adhésions_by_year['nb'] - adhésions_by_year[libelle_a_date]
del adhésions_by_year['nb']
adhésions_by_year.plot(kind="bar")
